## Different Models Stocks - Predicting Close Value from Open

In [5]:
# !pip install yfinance

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
%matplotlib inline

In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
df = df_data_1

NameError: name 'df_data_1' is not defined

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.value_counts()

In [ ]:
df["Date"] = pd.to_datetime(df['Date'])
df.head()

## Data Visualisation

In [ ]:
plt.plot(df['Date'],df['Close'])

In [ ]:
plt.plot(df['Date'],df['High'])

In [ ]:
plt.plot(df['Date'],df['Low'])

In [ ]:
!pip install mpl_finance

In [ ]:
!pip install --upgrade mplfinance

In [ ]:
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import pandas as pd
import matplotlib.dates as mpl_dates

plt.style.use('ggplot')

# Extracting Data for plotting
ohlc = df
ohlc['Date'] = pd.to_datetime(ohlc['Date'])
ohlc['Date'] = ohlc['Date'].apply(mpl_dates.date2num)
ohlc = ohlc.astype(float)

# Creating Subplots
fig, ax = plt.subplots()

candlestick_ohlc(ax, ohlc.values, width=0.6, colorup='green', colordown='red', alpha=0.8)

# Setting labels & titles
ax.set_xlabel('Date')
ax.set_ylabel('Price')
fig.suptitle('Daily Candlestick Chart of NIFTY50')

# Formatting Date
date_format = mpl_dates.DateFormatter('%d-%m-%Y')
ax.xaxis.set_major_formatter(date_format)
fig.autofmt_xdate()

fig.tight_layout()

plt.show()


## Data Splitting

In [ ]:
X = np.asanyarray(df[['Open']])
y = np.asanyarray(df['Close'])

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=4)

### Using Different Machine Learning Models

### 1. Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lre = LinearRegression()
lre.fit(x_train,y_train)
y_preds = lre.predict(x_test)
print("Y preds: ", y_preds[:5])
print("Y test: ", y_test[:5])

###  2. Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly